In [1]:
import pandas as pd

### `籌碼面資料調整`
> 1. 缺值補0，代表沒有持股。
>
> 2. 月平均持股率：日持股率進行平均，計算月平均。
>
> 3. 月平均週轉率：日週轉率進行平均，計算月平均。
>
> 4. 存成pickle檔

In [10]:
df_Invest = pd.read_pickle('籌碼面原始資料.pkl')
df_Invest.head(5)

,證券代碼,年月日,外資總投資股率%,投信持股率%,自營持股率%,合計持股率%,外資週轉率%,投信週轉率%,自營商週轉率%
0,1101 台泥,20100104,37.43,0.69,0.04,38.16,0.03,0.00,0.01
1,1102 亞泥,20100104,19.55,0.67,0.27,20.49,0.02,0.01,0.01
2,1103 嘉泥,20100104,4.15,0.04,0.11,4.30,0.02,0.00,0.03
3,1104 環泥,20100104,9.59,0.08,0.00,9.67,0.01,0.03,0.00
4,1108 幸福,20100104,1.72,0.00,0.00,1.72,0.00,0.00,0.03


In [11]:
df_Invest['年月日'] = pd.to_datetime(df_Invest['年月日'], format='%Y%m%d')
df_Invest['月份'] = df_Invest['年月日'].dt.to_period('M')

## 1.缺值補0，代表沒有持股。
df_Invest[['外資總投資股率%','投信持股率%','自營持股率%','合計持股率%','外資週轉率%','投信週轉率%','自營商週轉率%']] = df_Invest[['外資總投資股率%','投信持股率%','自營持股率%','合計持股率%','外資週轉率%','投信週轉率%','自營商週轉率%']].fillna(0)
df_Invest.head(5)

,證券代碼,年月日,外資總投資股率%,投信持股率%,自營持股率%,合計持股率%,外資週轉率%,投信週轉率%,自營商週轉率%,月份
0,1101 台泥,2010-01-04,37.43,0.69,0.04,38.16,0.03,0.00,0.01,2010-01
1,1102 亞泥,2010-01-04,19.55,0.67,0.27,20.49,0.02,0.01,0.01,2010-01
2,1103 嘉泥,2010-01-04,4.15,0.04,0.11,4.30,0.02,0.00,0.03,2010-01
3,1104 環泥,2010-01-04,9.59,0.08,0.00,9.67,0.01,0.03,0.00,2010-01
4,1108 幸福,2010-01-04,1.72,0.00,0.00,1.72,0.00,0.00,0.03,2010-01


In [12]:
## 2、3.算月平均值
columns_to_average = ['外資總投資股率%','投信持股率%','自營持股率%','合計持股率%','外資週轉率%','投信週轉率%','自營商週轉率%']
Average_Invest_df = df_Invest.groupby(['證券代碼', '月份'])[columns_to_average].mean().reset_index()
Average_Invest_df['月份'] = Average_Invest_df['月份'].astype(str)
Average_Invest_df = Average_Invest_df.sort_values(by=['月份','證券代碼'])
Average_Invest_df.rename(columns={'外資總投資股率%':'外資月平均持股率','投信持股率%':'投信月平均持股率','自營持股率%':'自營月平均持股率','合計持股率%':'合計月平均持股率','外資週轉率%':'外資月平均週轉率','投信週轉率%':'投信月平均週轉率','自營商週轉率%':'自營商月平均週轉率'}, inplace=True)
Average_Invest_df.head(5)

,證券代碼,月份,外資月平均持股率,投信月平均持股率,自營月平均持股率,合計月平均持股率,外資月平均週轉率,投信月平均週轉率,自營商月平均週轉率
0,1101 台泥,2010-01,37.2360,0.6740,0.0515,37.9625,0.1565,0.0155,0.0105
180,1102 亞泥,2010-01,19.5710,0.6815,0.2990,20.5500,0.0725,0.0025,0.0080
360,1103 嘉泥,2010-01,4.1395,0.0400,0.1030,4.2820,0.0195,0.0000,0.0070
540,1104 環泥,2010-01,9.6585,0.4445,0.0420,10.1435,0.0080,0.0185,0.0110
720,1108 幸福,2010-01,1.8805,0.0000,0.0000,1.8805,0.0195,0.0000,0.0020


In [14]:
Average_Invest_df.to_pickle('籌碼面數據_已整理.pkl')

### `營收面資料調整`
> 1. 營收資料放至月底，透過月底統一計算。
>
> 2. 若營收發布日有特殊狀況補增補的，即"營收發布日"不是"年月"的隔月，就刪掉。
>
> 3. 創新高/低(歷史)：為類別變數，以F補足缺失值(代表該月平淡、未創新高/低)。
>
> 4. 營收均線交叉：如果近3月累計營收平均大於近12月累計營收平均，就視為具有高成長性。
>
> 5. 如果有空值，代表成立時間未超過一年，不考慮。
>
> 6. 存成pickle檔。

In [15]:
df_Revenue = pd.read_pickle('營收面原始資料.pkl')
df_Revenue.head(5)

,證券代碼,年月,營收發布日,單月營收成長率％,累計營收成長率％,創新高/低(歷史),近 3月累計營收(千元),近12月累計營收(千元)
0,1101 台泥,201301,20130218,28.20,28.20,NaN,28600169.0,115666569.0
1,1102 亞泥,201301,20130208,55.10,55.10,H,17800272.0,65979660.0
2,1103 嘉泥,201301,20130218,-18.85,-18.85,NaN,1632470.0,4705949.0
3,1104 環泥,201301,20130208,52.50,52.50,H,1368871.0,4993816.0
4,1108 幸福,201301,20130218,68.33,68.33,H,1243520.0,4385587.0


In [16]:
## 1. 營收公告日放到月底，透過月底統一計算
df_Revenue['年月'] = pd.to_datetime(df_Revenue['年月'], format='%Y%m').dt.strftime('%Y-%m')
df_Revenue['營收發布日'] = pd.to_datetime(df_Revenue['營收發布日'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
df_Revenue['得知營收年月'] = df_Revenue['營收發布日'].str[:7]

## 2. 如果"營收發布日"不是"年月"的隔月，就刪掉
df_Revenue['年月'] = pd.to_datetime(df_Revenue['年月'], format='%Y-%m')
df_Revenue['得知營收年月'] = pd.to_datetime(df_Revenue['得知營收年月'], format='%Y-%m')
df_Revenue['對照用年月'] = df_Revenue['年月'] + pd.DateOffset(months=1)
df_Revenue = df_Revenue[df_Revenue['得知營收年月'] == df_Revenue['對照用年月']].copy() ## 確認條件是否符合

## 3.創新高/低(歷史)，以F補足缺失值(代表該月平淡、未創新高/低)。
df_Revenue['創新高/低(歷史)'] = df_Revenue['創新高/低(歷史)'].fillna('F')

## 4.營收均線交叉
df_Revenue['近3月營收均值高於12月營收均值'] = (df_Revenue['近 3月累計營收(千元)'] / 3 > df_Revenue['近12月累計營收(千元)'] / 12).astype(int)

## 5.刪除空值(代表成立時間未超過1年)
df_Revenue = df_Revenue.dropna()

## 6.刪除不必要欄位，存成pickle檔案
df_Revenue = df_Revenue[['證券代碼','年月','得知營收年月','單月營收成長率％','累計營收成長率％','創新高/低(歷史)','近3月營收均值高於12月營收均值']]
df_Revenue['年月'] = df_Revenue['年月'].dt.strftime('%Y-%m')
df_Revenue['得知營收年月'] = df_Revenue['得知營收年月'].dt.strftime('%Y-%m')
df_Revenue.head(5)

,證券代碼,年月,得知營收年月,單月營收成長率％,累計營收成長率％,創新高/低(歷史),近3月營收均值高於12月營收均值
0,1101 台泥,2013-01,2013-02,28.20,28.20,F,0
1,1102 亞泥,2013-01,2013-02,55.10,55.10,H,1
2,1103 嘉泥,2013-01,2013-02,-18.85,-18.85,F,1
3,1104 環泥,2013-01,2013-02,52.50,52.50,H,1
4,1108 幸福,2013-01,2013-02,68.33,68.33,H,1


In [17]:
df_Revenue.to_pickle('營收面數據_已整理.pkl')

### `技術面資料調整`
> 1. 缺值補0，就直接給0分概念 
>
> 2. 存成pickle檔。

In [18]:
df_Tech = pd.read_pickle('技術面原始資料.pkl')
df_Tech.head(5)

,證券代碼,年月,週轉率％_月,成交值比重％,股價淨值比-TSE,股價營收比-TEJ,股利殖利率-TSE,現金股利率,高低價差%
0,1101 台泥,201001,11.4999,0.576,1.28,4.57,4.42,4.42,21.1765
1,1539 巨庭,201001,2.7383,0.000,0.66,0.56,0.00,0.00,20.5952
2,1104 環泥,201001,9.3755,0.014,0.81,3.16,0.00,0.00,22.0000
3,1540 喬福,201001,5.3923,0.002,2.01,1.62,9.52,4.14,11.0783
4,1108 幸福,201001,9.7537,0.003,0.75,1.09,0.00,0.00,17.3184


In [19]:
df_Tech = df_Tech.sort_values(by=['年月','證券代碼'])
df_Tech['年月'] = pd.to_datetime(df_Tech['年月'].astype(str), format='%Y%m').dt.strftime('%Y-%m')
df_Tech[['週轉率％_月','成交值比重％','股價淨值比-TSE','股價營收比-TEJ','股利殖利率-TSE','現金股利率','高低價差%']] = df_Tech[['週轉率％_月','成交值比重％','股價淨值比-TSE','股價營收比-TEJ','股利殖利率-TSE','現金股利率','高低價差%']].fillna(0)
df_Tech.rename(columns={'週轉率％_月':'月週轉率','成交值比重％':'月成交值佔大盤比例','股價淨值比-TSE':'股價淨值比','股價營收比-TEJ':'股價營收比','股利殖利率-TSE':'股利殖利率','高低價差%':'高低價差比例'}, inplace=True)
df_Tech.head(5)

,證券代碼,年月,月週轉率,月成交值佔大盤比例,股價淨值比,股價營收比,股利殖利率,現金股利率,高低價差比例
0,1101 台泥,2010-01,11.4999,0.576,1.28,4.57,4.42,4.42,21.1765
599,1102 亞泥,2010-01,5.4953,0.319,1.26,8.56,6.93,5.77,18.0636
601,1103 嘉泥,2010-01,10.8984,0.015,0.68,4.50,0.00,0.00,18.7845
2,1104 環泥,2010-01,9.3755,0.014,0.81,3.16,0.00,0.00,22.0000
4,1108 幸福,2010-01,9.7537,0.003,0.75,1.09,0.00,0.00,17.3184


In [20]:
df_Tech.to_pickle('技術面數據_已整理.pkl')

### `過往報酬率資料調整`
> 1. 有缺失值刪除，因為其成立時間不夠，或是統計資料不足。
>
> 2. 原始都是抓月底最後一個交易日，所以就直接算是月底數據了，直接去除日。
>
> 3. 存成pickle檔。

In [21]:
df_Past_Return = pd.read_pickle('過往報酬率原始資料.pkl')
df_Past_Return.head(5)

,證券代碼,年月日,本週以來報酬率 %,本月以來報酬率 %,本季以來報酬率 %,本年以來報酬率 %,超額報酬(週)-大盤,超額報酬(月)-大盤,超額報酬(季)-大盤,超額報酬(年)-大盤
0,1101 台泥,20100129,-6.1320,-12.2058,-12.2058,-12.2058,-2.5134,-5.5172,-16.2980,-41.4070
1,1102 亞泥,20100129,-7.4809,-12.4276,-12.4276,-12.4276,-3.8623,-5.7390,-16.7719,-40.8415
2,1103 嘉泥,20100129,-7.3314,-12.7072,-12.7072,-12.7072,-3.7128,-6.0186,-10.0443,-41.2642
3,1104 環泥,20100129,-7.9319,-7.1428,-7.1428,-7.1428,-4.3133,-0.4542,-0.2581,-39.7751
4,1108 幸福,20100129,-6.8235,-11.5084,-11.5084,-11.5084,-3.2049,-4.8198,-7.2703,-30.4261


In [22]:
df_Past_Return['年月日'] = pd.to_datetime(df_Past_Return['年月日'].astype(str), format='%Y%m%d').dt.strftime('%Y-%m')
df_Past_Return = df_Past_Return.dropna()
df_Past_Return.head(5)

,證券代碼,年月日,本週以來報酬率 %,本月以來報酬率 %,本季以來報酬率 %,本年以來報酬率 %,超額報酬(週)-大盤,超額報酬(月)-大盤,超額報酬(季)-大盤,超額報酬(年)-大盤
0,1101 台泥,2010-01,-6.1320,-12.2058,-12.2058,-12.2058,-2.5134,-5.5172,-16.2980,-41.4070
1,1102 亞泥,2010-01,-7.4809,-12.4276,-12.4276,-12.4276,-3.8623,-5.7390,-16.7719,-40.8415
2,1103 嘉泥,2010-01,-7.3314,-12.7072,-12.7072,-12.7072,-3.7128,-6.0186,-10.0443,-41.2642
3,1104 環泥,2010-01,-7.9319,-7.1428,-7.1428,-7.1428,-4.3133,-0.4542,-0.2581,-39.7751
4,1108 幸福,2010-01,-6.8235,-11.5084,-11.5084,-11.5084,-3.2049,-4.8198,-7.2703,-30.4261


In [23]:
df_Past_Return.to_pickle('過往報酬率資料_已整理.pkl')

### `目標變數報酬率資料調整`
> 1. 有缺失值刪除，6438 迅德、6426 統新 分別在2021年1月22日、2021年3月26日，即月底最後交易日掛牌，故無當月資料。
>
> 2. 存成pickle檔。

In [24]:
df_Return = pd.read_pickle('目標變數報酬率原始資料.pkl')
df_Return.rename(columns={'報酬率％_月':'目標月報酬率%'}, inplace=True)
df_Return.head(5)

,證券代碼,年月,目標月報酬率%
0,1101 台泥,201301,2.5708
1,1102 亞泥,201301,0.5355
2,1103 嘉泥,201301,0.3609
3,1104 環泥,201301,6.2500
4,1108 幸福,201301,1.0481


In [25]:
df_Return = df_Return.dropna()
df_Return.to_pickle('目標變數報酬率資料_已整理.pkl')

### `合併所有資料`
> 1. 先合併現有因子表格。
>
> 2. 再調整下一個月份報酬率後進行合併。
>
> 3. 最後合併產業名稱。

In [26]:
import pandas as pd

In [27]:
df_Invest = pd.read_pickle('籌碼面數據_已整理.pkl')
df_Invest = df_Invest.rename(columns={"月份": "年月"})
df_Invest.head(5)

,證券代碼,年月,外資月平均持股率,投信月平均持股率,自營月平均持股率,合計月平均持股率,外資月平均週轉率,投信月平均週轉率,自營商月平均週轉率
0,1101 台泥,2010-01,37.2360,0.6740,0.0515,37.9625,0.1565,0.0155,0.0105
180,1102 亞泥,2010-01,19.5710,0.6815,0.2990,20.5500,0.0725,0.0025,0.0080
360,1103 嘉泥,2010-01,4.1395,0.0400,0.1030,4.2820,0.0195,0.0000,0.0070
540,1104 環泥,2010-01,9.6585,0.4445,0.0420,10.1435,0.0080,0.0185,0.0110
720,1108 幸福,2010-01,1.8805,0.0000,0.0000,1.8805,0.0195,0.0000,0.0020


In [28]:
df_Revenue = pd.read_pickle('營收面數據_已整理.pkl')
df_Revenue = df_Revenue.drop(columns=['年月'])
df_Revenue = df_Revenue.rename(columns={"得知營收年月": "年月"})
df_Revenue.head(5)

,證券代碼,年月,單月營收成長率％,累計營收成長率％,創新高/低(歷史),近3月營收均值高於12月營收均值
0,1101 台泥,2013-02,28.20,28.20,F,0
1,1102 亞泥,2013-02,55.10,55.10,H,1
2,1103 嘉泥,2013-02,-18.85,-18.85,F,1
3,1104 環泥,2013-02,52.50,52.50,H,1
4,1108 幸福,2013-02,68.33,68.33,H,1


In [29]:
df_Tech = pd.read_pickle('技術面數據_已整理.pkl')
df_Tech.head(5)

,證券代碼,年月,月週轉率,月成交值佔大盤比例,股價淨值比,股價營收比,股利殖利率,現金股利率,高低價差比例
0,1101 台泥,2010-01,11.4999,0.576,1.28,4.57,4.42,4.42,21.1765
599,1102 亞泥,2010-01,5.4953,0.319,1.26,8.56,6.93,5.77,18.0636
601,1103 嘉泥,2010-01,10.8984,0.015,0.68,4.50,0.00,0.00,18.7845
2,1104 環泥,2010-01,9.3755,0.014,0.81,3.16,0.00,0.00,22.0000
4,1108 幸福,2010-01,9.7537,0.003,0.75,1.09,0.00,0.00,17.3184


In [30]:
df_Past_Return = pd.read_pickle('過往報酬率資料_已整理.pkl')
df_Past_Return = df_Past_Return.rename(columns={"年月日": "年月"})
df_Past_Return.head(5)

,證券代碼,年月,本週以來報酬率 %,本月以來報酬率 %,本季以來報酬率 %,本年以來報酬率 %,超額報酬(週)-大盤,超額報酬(月)-大盤,超額報酬(季)-大盤,超額報酬(年)-大盤
0,1101 台泥,2010-01,-6.1320,-12.2058,-12.2058,-12.2058,-2.5134,-5.5172,-16.2980,-41.4070
1,1102 亞泥,2010-01,-7.4809,-12.4276,-12.4276,-12.4276,-3.8623,-5.7390,-16.7719,-40.8415
2,1103 嘉泥,2010-01,-7.3314,-12.7072,-12.7072,-12.7072,-3.7128,-6.0186,-10.0443,-41.2642
3,1104 環泥,2010-01,-7.9319,-7.1428,-7.1428,-7.1428,-4.3133,-0.4542,-0.2581,-39.7751
4,1108 幸福,2010-01,-6.8235,-11.5084,-11.5084,-11.5084,-3.2049,-4.8198,-7.2703,-30.4261


In [31]:
df_Return = pd.read_pickle('目標變數報酬率資料_已整理.pkl')
df_Return["年月"] = pd.to_datetime(df_Return["年月"].astype(str), format="%Y%m")
df_Return['對照用年月'] = (df_Return['年月'] + pd.DateOffset(months=-1)).dt.strftime("%Y-%m")
df_Return = df_Return.drop(columns=['年月'])
df_Return

,證券代碼,目標月報酬率%,對照用年月
0,1101 台泥,2.5708,2012-12
1,1102 亞泥,0.5355,2012-12
2,1103 嘉泥,0.3609,2012-12
3,1104 環泥,6.2500,2012-12
4,1108 幸福,1.0481,2012-12
...,...,...,...
126302,9944 新麗,0.7445,2025-01
126303,9945 潤泰新,0.4726,2025-01
126304,9946 三發地產,23.6280,2025-01
126305,9955 佳龍,4.8611,2025-01


In [32]:
## 合併所有變數
df_Invest_Revenue = pd.merge(df_Invest, df_Revenue, on=["證券代碼", "年月"], how="inner")
df_Invest_Revenue_Tech = pd.merge(df_Invest_Revenue, df_Tech, on=["證券代碼", "年月"], how="inner")
df_Invest_Revenue_Tech_Past = pd.merge(df_Invest_Revenue_Tech, df_Past_Return, on=["證券代碼", "年月"], how="inner")
df_Invest_Revenue_Tech_Past = df_Invest_Revenue_Tech_Past.rename(columns={"年月": "對照用年月"})
df_Invest_Revenue_Tech_Past.head(5)

,證券代碼,對照用年月,外資月平均持股率,投信月平均持股率,自營月平均持股率,合計月平均持股率,外資月平均週轉率,投信月平均週轉率,自營商月平均週轉率,單月營收成長率％,...,現金股利率,高低價差比例,本週以來報酬率 %,本月以來報酬率 %,本季以來報酬率 %,本年以來報酬率 %,超額報酬(週)-大盤,超額報酬(月)-大盤,超額報酬(季)-大盤,超額報酬(年)-大盤
0,1101 台泥,2013-02,37.186923,1.975385,0.043077,39.207692,0.131538,0.014615,0.012308,28.20,...,4.99,9.0226,-1.0403,-4.6365,-2.1849,-2.1849,1.1103,-5.2474,-4.1922,9.2376
1,1102 亞泥,2013-02,18.820000,0.953077,0.005385,19.776923,0.086923,0.001538,0.003846,55.10,...,6.04,3.8615,0.2711,-1.4648,-0.9371,-0.9371,3.5615,-2.0757,-5.1297,10.3943
2,1103 嘉泥,2013-02,4.946154,0.020769,0.020000,4.985385,0.006154,0.000000,0.000000,-18.85,...,1.39,2.8777,0.0001,0.3598,0.7220,0.7220,1.6332,-0.2511,0.3021,6.8882
3,1104 環泥,2013-02,9.822308,0.619231,0.046154,10.486154,0.010769,0.003077,0.002308,52.50,...,3.35,11.1765,0.8450,5.2939,11.8748,11.8748,6.6184,4.6830,15.5399,28.1627
4,1108 幸福,2013-02,2.031538,0.000000,0.000000,2.032308,0.000769,0.000000,0.000000,68.33,...,0.00,8.7407,2.5752,6.2223,7.3356,7.3356,5.5462,5.6114,9.0781,23.4587


In [33]:
## 合併目標變數
df = pd.merge(df_Invest_Revenue_Tech_Past, df_Return, on=["證券代碼", "對照用年月"], how="inner")
df = df.rename(columns={"目標月報酬率%": "下一個月報酬率%"})
df.head(5)

,證券代碼,對照用年月,外資月平均持股率,投信月平均持股率,自營月平均持股率,合計月平均持股率,外資月平均週轉率,投信月平均週轉率,自營商月平均週轉率,單月營收成長率％,...,高低價差比例,本週以來報酬率 %,本月以來報酬率 %,本季以來報酬率 %,本年以來報酬率 %,超額報酬(週)-大盤,超額報酬(月)-大盤,超額報酬(季)-大盤,超額報酬(年)-大盤,下一個月報酬率%
0,1101 台泥,2013-02,37.186923,1.975385,0.043077,39.207692,0.131538,0.014615,0.012308,28.20,...,9.0226,-1.0403,-4.6365,-2.1849,-2.1849,1.1103,-5.2474,-4.1922,9.2376,-1.4454
1,1102 亞泥,2013-02,18.820000,0.953077,0.005385,19.776923,0.086923,0.001538,0.003846,55.10,...,3.8615,0.2711,-1.4648,-0.9371,-0.9371,3.5615,-2.0757,-5.1297,10.3943,-1.4866
2,1103 嘉泥,2013-02,4.946154,0.020769,0.020000,4.985385,0.006154,0.000000,0.000000,-18.85,...,2.8777,0.0001,0.3598,0.7220,0.7220,1.6332,-0.2511,0.3021,6.8882,-5.7348
3,1104 環泥,2013-02,9.822308,0.619231,0.046154,10.486154,0.010769,0.003077,0.002308,52.50,...,11.1765,0.8450,5.2939,11.8748,11.8748,6.6184,4.6830,15.5399,28.1627,-0.2793
4,1108 幸福,2013-02,2.031538,0.000000,0.000000,2.032308,0.000769,0.000000,0.000000,68.33,...,8.7407,2.5752,6.2223,7.3356,7.3356,5.5462,5.6114,9.0781,23.4587,1.1158


In [34]:
df_Industry = pd.read_excel('公司產業別.xlsx')
df_Industry.head(5)

,證券代碼,TSE產業名
0,1101 台泥,M1100 水泥工業
1,1102 亞泥,M1100 水泥工業
2,1103 嘉泥,M1100 水泥工業
3,1104 環泥,M1100 水泥工業
4,1107 建台,M9900 其他


In [35]:
## 合併公司產業別
df = pd.merge(df, df_Industry,  on=["證券代碼"], how="left")
df

,證券代碼,對照用年月,外資月平均持股率,投信月平均持股率,自營月平均持股率,合計月平均持股率,外資月平均週轉率,投信月平均週轉率,自營商月平均週轉率,單月營收成長率％,...,本週以來報酬率 %,本月以來報酬率 %,本季以來報酬率 %,本年以來報酬率 %,超額報酬(週)-大盤,超額報酬(月)-大盤,超額報酬(季)-大盤,超額報酬(年)-大盤,下一個月報酬率%,TSE產業名
0,1101 台泥,2013-02,37.186923,1.975385,0.043077,39.207692,0.131538,0.014615,0.012308,28.20,...,-1.0403,-4.6365,-2.1849,-2.1849,1.1103,-5.2474,-4.1922,9.2376,-1.4454,M1100 水泥工業
1,1102 亞泥,2013-02,18.820000,0.953077,0.005385,19.776923,0.086923,0.001538,0.003846,55.10,...,0.2711,-1.4648,-0.9371,-0.9371,3.5615,-2.0757,-5.1297,10.3943,-1.4866,M1100 水泥工業
2,1103 嘉泥,2013-02,4.946154,0.020769,0.020000,4.985385,0.006154,0.000000,0.000000,-18.85,...,0.0001,0.3598,0.7220,0.7220,1.6332,-0.2511,0.3021,6.8882,-5.7348,M1100 水泥工業
3,1104 環泥,2013-02,9.822308,0.619231,0.046154,10.486154,0.010769,0.003077,0.002308,52.50,...,0.8450,5.2939,11.8748,11.8748,6.6184,4.6830,15.5399,28.1627,-0.2793,M1100 水泥工業
4,1108 幸福,2013-02,2.031538,0.000000,0.000000,2.032308,0.000769,0.000000,0.000000,68.33,...,2.5752,6.2223,7.3356,7.3356,5.5462,5.6114,9.0781,23.4587,1.1158,M1100 水泥工業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123242,9944 新麗,2024-12,2.172727,0.000000,0.003182,2.174545,0.011364,0.000000,0.000000,-0.11,...,1.5000,-0.9757,-0.4903,3.2050,0.3683,-4.4461,-4.1375,-25.2625,-0.7388,M9900 其他
123243,9945 潤泰新,2024-12,13.764091,0.010000,0.025455,13.803182,0.050909,0.000000,0.001364,-2.27,...,-0.3484,-4.2412,-12.3596,17.3089,-0.4409,-7.7117,-16.0068,-11.1587,-1.3987,M9900 其他
123244,9946 三發地產,2024-12,4.750000,0.000000,0.010000,4.761818,0.029545,0.000000,0.000000,85.36,...,0.8929,-11.0238,-21.3917,13.0070,-1.7961,-14.4942,-25.0389,-15.4606,-7.3009,M2500 建材營造
123245,9955 佳龍,2024-12,1.658636,0.000000,0.005909,1.666364,0.102727,0.000000,0.000000,-1.20,...,0.1757,-6.2500,-21.7034,16.8021,1.4320,-9.7205,-25.3507,-11.6655,1.0526,M3500 綠能環保


In [36]:
df.to_pickle('BacktestData.pkl')